In [ ]:
import torch

import arviz as az
import matplotlib.pyplot as plt

from autoemulate.simulations.epidemic import Epidemic
from autoemulate.core.compare import AutoEmulate
from autoemulate.calibration.bayes import BayesianCalibration
from autoemulate.emulators import GaussianProcess
import pyro 


In [ ]:
random_seed = 42

# Bayesian calibration

Bayesian calibration is a method for estimating which input parameters were most likely to produce observed data. An advantage over other calibration methods is that it returns a probability distribution over the input parameters rather than just point estimates.

Performing Bayesian calibration requires:
- a fitted emulator
- observations associated with the simulator output



## 1. Simulate data and train an emulator

In this example, we'll use the `Epidemic` simulator, which returns the peak infection rate given two input parameters, `beta`(the transimission rate per day) and `gamma` (the recovery rate per day).

In [ ]:
simulator = Epidemic(log_level="error")
x = simulator.sample_inputs(1000, random_seed=random_seed)
y = simulator.forward_batch(x)

Below we plot the simulated today. The peak infection rate is higher when the transmission rate increases and the recovery rate decreases.

In [ ]:
transmission_rate = x[:, 0]
recovery_rate = x[:, 1]

plt.scatter(transmission_rate, recovery_rate, c=y, cmap='viridis')
plt.xlabel('Transmission rate (beta)')
plt.ylabel('Recovery rate (gamma)')
plt.colorbar(label="Peak infection rate")
plt.show

For the purposes of this tutorial, we will restrict the emulator choice to `GaussianProcess` with default hyperparameters.

In [ ]:
ae = AutoEmulate(
    x, 
    y, 
    models=[GaussianProcess], 
    log_level="error", 
    random_seed=random_seed,
    model_tuning=False
    )

We can verify that the fitted emulator performs well on both the train and test data.

In [ ]:
ae.summarise()

In [ ]:
gp = ae.best_result().model

## 2. Calibrate

Calibration requires at least one or multiple observations. These can come from running experiments or from the literature.

Below we pick the initial parameter values we want to infer and simulate the output. We then add noise to generate 100 "observations".

In [ ]:
true_beta = 0.2
true_gamma = 0.1 

# simulator expects inputs of shape [1, number of inputs]
params = torch.tensor([true_beta, true_gamma]).view(1, -1)
true_infection_rate = simulator.forward(params)

n_obs = 100
stdev = 0.05
noise = torch.normal(mean=0, std=stdev, size=(n_obs,))
observed_infection_rates = true_infection_rate[0] + noise

print("Observed infection rates:", observed_infection_rates.numpy().round(3))

We set up the `BayesianCalibration` object with the trained emulator, the simulator parameter ranges and the "observed" data we simulated above. The underlying probabilistic model assumes the observations are drawn from a Gaussian distribution with the mean predicted by the emulator. We also have to specify the `observation_noise` of this Gaussian likelihood.

In [ ]:
observations = {"infection_rate": observed_infection_rates}

bc = BayesianCalibration(
    gp, 
    simulator.parameters_range, 
    observations, 
    # specify noise as variance
    observation_noise=stdev**2
)

Run MCMC using the NUTS sampler.

In [ ]:
pyro.set_rng_seed(random_seed)

mcmc = bc.run_mcmc(
    warmup_steps=250, 
    num_samples=1000,
    sampler='nuts',
    num_chains=2,
    
)

The above returns the Pyro MCMC object which has a number of useful methods associated with it. One can access all the posterior samples using `mcmc.get_samples()` or just the summary statistics using `mcmc.summary()`.

In [ ]:
mcmc.summary()

## 3. Plotting with Arviz

The `BayesianCalibrator.to_arviz` method converts the `mcmc` object so that it is compatible with the Arviz plotting library. Using Arviz makes it very easy to produce all the standard plots of the calibration results as well as MCMC diagnostics.

In [ ]:
az_data = bc.to_arviz(mcmc, posterior_predictive=True)

The 2D KDE of the posterior distribution can also be visualized.

In [ ]:
_ = az.plot_pair(az_data, kind='kde')

The posterior predictive mean and posterior predictive samples can be plotted alongside the observed data.

In [ ]:
_ = az.plot_ppc(az_data, kind='kde')

To visualize the posterior distribution, the samples from the posterior distribution can be viewed as a trace (right-hand plots) with 1D KDEs for each chain for each variable (left-hand plots).

In [ ]:
_ = az.plot_trace(az_data, figsize=(20, 8))